In [124]:
WORKSPACE_PATH = 'workspace'
ANNOTATION_PATH = WORKSPACE_PATH+'/annotations'
IMAGE_PATH = WORKSPACE_PATH+'/images'
MODEL_PATH = WORKSPACE_PATH+'/models'

In [125]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np
import cv2


In [126]:
import json

def json_read(data_path):
    with open(data_path, 'r') as data:
        data = json.load(data)
    return data

In [127]:
XML_NAME = '/instances_default.json'
DATA_PATH = ANNOTATION_PATH+'/original'+ XML_NAME
data = json_read(DATA_PATH)

In [128]:
import os

def rename_images(IMAGE_PATH):
    index=1
    img  = os.listdir(IMAGE_PATH)
    for i in img:
        new_name = f"{index:012d}"+'.jpeg'
        old_path = os.path.join(IMAGE_PATH, i)
        new_path = os.path.join(IMAGE_PATH, new_name)
        os.rename(old_path, new_path)
        index += 1

def rename_images_in_annotation():
    index=1
    for img in data['images']:
        new_name = f"{index:012d}"+'.jpeg'
        img['file_name'] = new_name 
        index += 1

rename_images(IMAGE_PATH+'/original')
rename_images_in_annotation()

with open(ANNOTATION_PATH+'/original' + XML_NAME, 'w') as destination_file:
    json.dump(data, destination_file, indent=2)  

In [129]:
images = []
bounding_boxes = []
labels = []
image_height = 1920
image_width = 1080

for image in data['images']:
    one_image = []
    one_image.append(image['id'])
    one_image.append(image['file_name'])
    images.append(one_image)

for annotation in data['annotations']:
    one_box = []
    box_and_id = []
    one_box.append(annotation['bbox'])
    #one_box.append(annotation['id'])
    #one_box.append(annotation['image_id'])
    box_and_id.append(one_box)
    box_and_id.append(annotation['image_id'])
    bounding_boxes.append(box_and_id)

print(images)
print(bounding_boxes)
print(bounding_boxes[2][0])


[[1, '000000000001.jpeg'], [2, '000000000002.jpeg'], [3, '000000000003.jpeg'], [4, '000000000004.jpeg'], [5, '000000000005.jpeg'], [6, '000000000006.jpeg'], [7, '000000000007.jpeg'], [8, '000000000008.jpeg'], [9, '000000000009.jpeg'], [10, '000000000010.jpeg'], [11, '000000000011.jpeg'], [12, '000000000012.jpeg'], [13, '000000000013.jpeg'], [14, '000000000014.jpeg'], [15, '000000000015.jpeg'], [16, '000000000016.jpeg'], [17, '000000000017.jpeg']]
[[[[39.37, 366.56, 481.39, 304.52]], 1], [[[45.75, 429.5, 459.55, 41.3]], 1], [[[44.68, 472.77, 459.55, 41.3]], 1], [[[43.86, 518.93, 459.54, 41.3]], 1], [[[30.05, 669.42, 492.05, 296.7]], 1], [[[45.19, 703.01, 458.15, 44.2]], 1], [[[42.64, 749.76, 226.95, 41.65]], 1], [[[273.84, 751.46, 231.2, 39.95]], 1], [[[45.19, 794.81, 223.55, 40.8]], 1], [[[277.24, 811.81, 19.55, 22.95]], 1], [[[356.29, 812.66, 19.55, 22.95]], 1], [[[427.69, 811.81, 20.4, 22.95]], 1], [[[45.19, 839.01, 128.35, 44.2]], 1], [[[172.69, 838.16, 96.9, 46.75]], 1], [[[379.24,

In [130]:
def resize_imgaes(input_path, output_path, height, width):
    image =  cv2.imread(input_path)
    resized_image = cv2.resize(image, (height, width))
    cv2.imwrite(output_path, resized_image)    

def resize_bounding_boxes_and_area(input_path,bounding_boxes, height, width):
    image =  cv2.imread(input_path)
    height_ratio = height / image.shape[0]
    width_ratio = width / image.shape[1]
    
    for box in bounding_boxes:
        resized_boxes = []
        resized_area = 0

        x = np.round(box[0]*width_ratio,2)
        y = np.round(box[1]*height_ratio,2)
        x_width = np.round(box[2]*width_ratio,2)
        y_height = np.round(box[3]*height_ratio,2)
        resized_boxes.append([x, y, x_width, y_height])

        resized_area = x_width * y_height

    return x, y, x_width, y_height, resized_area


In [131]:
import os

height = 224
width = 224
OUTPUT_PATH_IMAGES = IMAGE_PATH + '/' + str(height) + 'x' + str(width)
OUTPUTPATH_ANNOTATIONS = ANNOTATION_PATH + '/' + str(height) + 'x' + str(width)

os.makedirs(OUTPUT_PATH_IMAGES, exist_ok=True)
os.makedirs(OUTPUTPATH_ANNOTATIONS, exist_ok=True)

#Copy JSON Doc in new ANNOTATIONPATH
with open(DATA_PATH, 'r') as source_file:
    data = json.load(source_file)
with open(OUTPUTPATH_ANNOTATIONS + XML_NAME, 'w') as destination_file:
    json.dump(data, destination_file, indent=2)    


In [132]:
#Resize Images
for img in range(len(images)):
    resize_imgaes(IMAGE_PATH+'/original'+ '/' +images[img][1], OUTPUT_PATH_IMAGES+ '/' +images[img][1], height, width)


with open(OUTPUTPATH_ANNOTATIONS + XML_NAME, 'r') as json_data:
    data = json.load(json_data)


#Resize BBox and area
for i, annotation in enumerate(data['annotations']):
    calculate_resize = resize_bounding_boxes_and_area(IMAGE_PATH+'/original'+ '/' +images[0][1],bounding_boxes[i][0], height, width)
    new_box = calculate_resize[0],calculate_resize[1],calculate_resize[2],calculate_resize[3]
    new_area = calculate_resize[4]
    annotation['bbox'] = new_box
    annotation['area'] = new_area

#Change height/width
for image in data['images']:
    image['height']=height
    image['width']=width

with open(OUTPUTPATH_ANNOTATIONS + XML_NAME, 'w') as destination_file:
    json.dump(data, destination_file, indent=2)    

In [133]:
import keras_cv

pretrained_model = keras_cv.models.YOLOV8Detector.from_preset(
    "yolo_v8_m_pascalvoc", bounding_box_format="xywh"
)

In [134]:
SPLIT_RATIO = 0.2
BATCH_SIZE = 4
LEARNING_RATE = 0.001
EPOCH = 5
GLOBAL_CLIPNORM = 10.0

In [135]:
class_ids = []

for i in data['categories']:
    new_name = i['name']
    class_ids.append(new_name)

class_mapping = dict(zip(range(len(class_ids)), class_ids))

In [145]:
OUTPUT_PATH_IMAGES
#klassen 42
#imgaes 17
image_paths = []
bbox = []
classes = []
count = 0

    
def count_images(path):
    try:
        entries = os.listdir(path)
        file_count = sum(os.path.isfile(os.path.join(path, entry)) for entry in entries)
        return file_count
    except Exception as e:
        print(f"Fehler beim Zählen der Dateien: {e}")
        return None

number_images = count_images(OUTPUT_PATH_IMAGES)


In [146]:
for image in data['images']:
    name = image['file_name']
    image_paths.append(OUTPUT_PATH_IMAGES + '/' + name)
    #filename.append(image[1])
print(len(image_paths))
print(len(image_paths[0]))
print(image_paths)

17
42
['workspace/images/224x224/000000000001.jpeg', 'workspace/images/224x224/000000000002.jpeg', 'workspace/images/224x224/000000000003.jpeg', 'workspace/images/224x224/000000000004.jpeg', 'workspace/images/224x224/000000000005.jpeg', 'workspace/images/224x224/000000000006.jpeg', 'workspace/images/224x224/000000000007.jpeg', 'workspace/images/224x224/000000000008.jpeg', 'workspace/images/224x224/000000000009.jpeg', 'workspace/images/224x224/000000000010.jpeg', 'workspace/images/224x224/000000000011.jpeg', 'workspace/images/224x224/000000000012.jpeg', 'workspace/images/224x224/000000000013.jpeg', 'workspace/images/224x224/000000000014.jpeg', 'workspace/images/224x224/000000000015.jpeg', 'workspace/images/224x224/000000000016.jpeg', 'workspace/images/224x224/000000000017.jpeg']


In [138]:
ann=[]

for image in data['images']:
    ann=[]
    for annotation in data['annotations'] :
        if image['id'] == annotation['image_id']:
            ann.append(annotation['bbox'])
    bbox.append(ann)
    #filename.append(image[1])
print(len(bbox))
print(len(bbox[0]))
print(bbox)

17
42
[[(11.48, 60.11, 140.41, 49.94), (13.34, 70.43, 134.04, 6.77), (13.03, 77.53, 134.04, 6.77), (12.79, 85.1, 134.03, 6.77), (8.76, 109.77, 143.51, 48.65), (13.18, 115.28, 133.63, 7.25), (12.44, 122.95, 66.19, 6.83), (79.87, 123.23, 67.43, 6.55), (13.18, 130.33, 65.2, 6.69), (80.86, 133.12, 5.7, 3.76), (103.92, 133.26, 5.7, 3.76), (124.74, 133.12, 5.95, 3.76), (13.18, 137.58, 37.44, 7.25), (50.37, 137.44, 28.26, 7.67), (110.61, 138.14, 36.69, 7.11), (12.68, 146.36, 66.44, 6.83), (80.61, 146.09, 67.19, 7.39), (117.06, 153.19, 5.21, 3.62), (10.25, 157.7, 141.86, 22.94), (12.15, 163.79, 70.42, 7.45), (80.65, 163.6, 23.78, 7.65), (103.55, 163.5, 44.09, 7.74), (12.32, 171.34, 15.15, 7.16), (26.76, 171.25, 23.43, 7.26), (48.45, 171.34, 98.84, 7.36), (10.39, 179.06, 140.69, 35.66), (11.83, 199.04, 70.04, 6.85), (80.51, 198.94, 23.55, 6.86), (103.11, 198.94, 42.68, 6.93), (11.72, 206.89, 15.21, 6.78), (26.05, 206.73, 23.55, 6.62), (48.44, 206.73, 97.58, 6.86), (116.89, 187.7, 5.74, 3.39), (

In [139]:
ann=[]

for image in data['images']:
    ann=[]
    for annotation in data['annotations'] :
        if image['id'] == annotation['image_id']:
            ann.append(annotation['category_id'])
    classes.append(ann)
    #filename.append(image[1])
print(classes)
print(len(classes[0]))


[[1, 2, 3, 3, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 39, 39, 7, 8, 5, 6, 9, 10], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 39, 39, 7, 8, 5, 6, 9, 10, 2, 3, 4], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 39, 39, 7, 8, 5, 6, 9, 10, 2, 3, 4], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 39, 39, 7, 8, 5, 6, 9, 10, 2, 3, 4], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 39, 39, 7, 8, 5, 6, 9, 10, 2, 3, 4], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 28, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 39, 39, 7, 8, 5, 6, 9, 10, 2, 3, 4], [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,

In [147]:
ragged_bbox = tf.ragged.constant(bbox)
ragged_classes = tf.ragged.constant(classes)
ragged_image_paths = tf.ragged.constant(OUTPUT_PATH_IMAGES)

data = tf.data.Dataset.from_tensor_slices((ragged_image_paths, ragged_classes, ragged_bbox))


ValueError: Unbatching a tensor is only supported for rank >= 1

In [148]:
import os
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow import keras

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization


Rename images

In [162]:
def change_image_names(xml_file, start_number=1, file_extension=".jpeg"):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    for image_element in root.iter('image'):
        # Generate the new image name with an ascending number
        new_image_name = f"{start_number:012d}{file_extension}"

        # Update the image name in the XML file
        image_element.set("name", new_image_name)

        start_number += 1

    # Save the modified XML file
    tree.write(xml_file)

# Beispielaufruf
xml_file = ANNOTATION_PATH + '/original/annotations.xml'
change_image_names(xml_file)

In [163]:
def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()

    image_name = root.find("name").text
    image_path = os.path.join(OUTPUT_PATH_IMAGES, image_name)

    boxes = []
    classes = []
    for obj in root.iter("object"):
        cls = obj.find("name").text
        classes.append(cls)

        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)
        boxes.append([xmin, ymin, xmax, ymax])

    class_ids = [
        list(class_mapping.keys())[list(class_mapping.values()).index(cls)]
        for cls in classes
    ]
    return image_path, boxes, class_ids


image_paths = []
bbox = []
classes = []
for xml_file in tqdm(r"C:\Users\Tristan\studium\TiProjekt\BBArchComparison\Tensorflow\workspace\annotations\original\annotations.xml"):#ANNOTATION_PATH + '/original/annotations.xml'):
    image_path, boxes, class_ids = parse_annotation(xml_file)
    image_paths.append(image_path)
    bbox.append(boxes)
    classes.append(class_ids)


  0%|          | 0/109 [00:00<?, ?it/s]

FileNotFoundError: [Errno 2] No such file or directory: 'C'